In [1]:
# Cell 1: ultralytics 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
# Cell 2: Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cell 3: 추론 대상 압축 해제
import os, zipfile

zip_path = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/unlabeled_seatbelt"
unzip_output = "/content/unlabeled_infer"
os.makedirs(unzip_output, exist_ok=True)

# zip 파일 하나 혹은 여러 개 모두 해제
for fname in os.listdir(zip_path):
    if fname.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(zip_path, fname), 'r') as zip_ref:
            zip_ref.extractall(unzip_output)

print(f"✅ 압축 해제 완료: {unzip_output}")

✅ 압축 해제 완료: /content/unlabeled_infer


In [4]:
import os
from glob import glob

# 압축 풀린 루트 확인
print("루트 폴더 내 디렉토리:")
print(os.listdir("/content/unlabeled_infer"))

# 이미지 경로 확인 (재귀적으로 탐색)
img_files = glob("/content/unlabeled_infer/**/*.jpg", recursive=True)
print(f"총 이미지 수: {len(img_files)}")

# 상위 몇 개만 출력
for path in img_files[:5]:
    print(path)

루트 폴더 내 디렉토리:
['train.txt', 'obj.data', 'obj_train_data', 'obj.names']
총 이미지 수: 10000
/content/unlabeled_infer/obj_train_data/20240524_002240_NOR_0035_0.jpg
/content/unlabeled_infer/obj_train_data/20240524_105956_NOR_0079_0.jpg
/content/unlabeled_infer/obj_train_data/20240523_233158_NOR_0074_0.jpg
/content/unlabeled_infer/obj_train_data/20240523_230322_NOR_0009_0.jpg
/content/unlabeled_infer/obj_train_data/20240523_203332_NOR_0002_0.jpg


In [5]:
import os
import shutil
from glob import glob
from tqdm import tqdm

# 1. 설정
model_path = "/content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/best.pt"
img_input_dir = "/content/unlabeled_infer/obj_train_data"  # 추론 대상 원본 이미지 폴더
save_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result"  # ✅ 저장 위치 (Drive)

# 폴더 생성
os.makedirs(save_dir, exist_ok=True)

# 2. 모델 로드 + 추론
from ultralytics import YOLO
model = YOLO(model_path)

results = model.predict(
    source=img_input_dir,
    conf=0.5,
    save_txt=True,
    # save_conf=True,
    save=True,
    imgsz=640,
    project=save_dir,
    name="seatbelt_yolo_export",
    # exist_ok=True
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/10000 /content/unlabeled_infer/obj_train_data/20240523_202952_NOR_0022_0.jpg: 384x640 1 Seatbelt, 42.2ms
image 2/10000 /content/unlabeled

KeyboardInterrupt: 

In [ ]:
# Cell 3: 누락된 .txt 라벨 자동 생성
import glob

result_dir = os.path.join(save_dir, "seatbelt_yolo_export")
label_dir = os.path.join(result_dir, "labels")
image_dir = result_dir

# 이미지 리스트 수집
img_files = glob.glob(os.path.join(image_dir, "*.jpg"))

# 누락된 라벨 생성
for img_path in img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(label_dir, base + ".txt")
    if not os.path.exists(txt_path):
        open(txt_path, 'w').close()

print("✅ 누락된 .txt 파일 생성 완료")

✅ 누락된 .txt 파일 생성 완료


In [ ]:
# Cell 4: .jpg + .txt 압축 (Drive 내부 저장)
from shutil import make_archive
import shutil

# 압축 대상 경로 준비 (모아 담기)
export_dir = os.path.join(save_dir, "export_zip_temp")
os.makedirs(export_dir, exist_ok=True)

# 이미지와 txt 복사
for img_path in img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(label_dir, base + ".txt")

    shutil.copy(img_path, os.path.join(export_dir, base + ".jpg"))
    shutil.copy(txt_path, os.path.join(export_dir, base + ".txt"))

# 압축
zip_path = os.path.join(save_dir, "seatbelt_yolo_export.zip")
make_archive(zip_path.replace(".zip", ""), 'zip', export_dir)

print("✅ 압축 완료:", zip_path)

✅ 압축 완료: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export.zip


In [ ]:
# Cell 5: 다운로드
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>